In this chapter, I perform a simple analysis of relation between trade with China and debt to gdp ratio for OECD countries.

# 0 Prepare environment

In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from linearmodels.panel import PanelOLS
import requests
import time
from fredapi import Fred
from TradeDebtAnalyzer import TradeDebtAnalyzer


In [3]:
import os
os.getcwd()

'c:\\Users\\trend\\Documents\\GitHub\\graspp_2025_spring\\notebooks\\assignment_individual\\HW1'

# 1.1 Download Data
I suppressed the download function and used the already downloaded data in the following.

In [3]:
#df_final = TradeDebtAnalyzer.download_data()
# def download_data():
#         api_key_un = "53408856aa4441c8af4cf29b62aab37a"

#         def get_uncomtrade_data(subscription_key: str, params: dict) -> pd.DataFrame:
#             base_url = f"https://comtradeapi.un.org/data/v1/get/{params['typeCode']}/{params['freqCode']}/{params['clCode']}"
#             query_params = {"subscription-key": subscription_key, **params}
#             try:
#                 r = requests.get(base_url, params=query_params, timeout=60)
#                 r.raise_for_status()
#                 data = r.json()
#                 return pd.DataFrame(data["data"]) if data.get("data") else pd.DataFrame()
#             except (requests.RequestException, ValueError) as err:
#                 print(f"Request failed: {err}")
#                 return pd.DataFrame()

#         country_list = [36, 40, 56, 124, 208, 233, 251, 300, 348, 352,
#                         372, 392, 440, 554, 703, 724, 752, 826, 842]
#         dfs = []
#         for country in country_list:
#             for year in range(2010, 2024):
#                 query_cty = {
#                     "typeCode": "C", "freqCode": "A", "clCode": "HS",
#                     "period": f"{year}", "reporterCode": f"{country}",
#                     "partnerCode": "156", "partner2Code": "0",
#                     "flowCode": "M", "maxrecords": "1000", "cmdCode": "TOTAL"
#                 }
#                 time.sleep(2)
#                 df_cty = get_uncomtrade_data(api_key_un, query_cty)
#                 time.sleep(2)
#                 if not df_cty.empty:
#                     print(df_cty.shape, country, year)
#                     dfs.append(df_cty)

#         df = pd.concat(dfs, ignore_index=True)
#         df_all = df[df['motCode'] == 0]
#         df_all_sum = df_all.groupby(['period','reporterCode'])['primaryValue'].sum().reset_index()
#         df_trade = df_all_sum[df_all_sum['reporterCode'] != 528]

#         fred = Fred(api_key='a8be37f8d6242d8ee6601727cf359c50')

#         debt_series = {
#             '36': 'DEBTTLAUA188A', '40': 'DEBTTLATA188A', '56': 'DEBTTLBEA188A',
#             '124': 'DEBTTLCAA188A', '208': 'DEBTTLDEA188A', '233': 'DEBTTLEEA188A',
#             '251': 'DEBTTLFRA188A', '300': 'DEBTTLGRA188A', '348': 'DEBTTLHUA188A',
#             '352': 'DEBTTLISA188A', '372': 'DEBTTLIEA188A', '392': 'DEBTTLJPA188A',
#             '440': 'DEBTTLLTA188A', '554': 'DEBTTLNZA188A', '703': 'DEBTTLSKA188A',
#             '724': 'DEBTTLESA188A', '752': 'DEBTTLSEA188A', '826': 'DEBTTLGBA188A', '842': 'DEBTTLUSA188A'
#         }

#         gdp_pc_series = {
#             '36': 'NYGDPPCAPKDAUS', '40': 'NYGDPPCAPKDAUT', '56': 'NYGDPPCAPKDBEL',
#             '124': 'NYGDPPCAPKDCAN', '208': 'NYGDPPCAPKDDNK', '233': 'NYGDPPCAPKDEST',
#             '251': 'NYGDPPCAPKDFRA', '300': 'NYGDPPCAPKDGRC', '348': 'NYGDPPCAPKDHUN',
#             '352': 'NYGDPPCAPKDISL', '372': 'NYGDPPCAPKDIRL', '392': 'NYGDPPCAPKDJPN',
#             '440': 'NYGDPPCAPKDLTU', '554': 'NYGDPPCAPKDNZL', '703': 'NYGDPPCAPKDSVK',
#             '724': 'NYGDPPCAPKDESP', '752': 'NYGDPPCAPKDSWE', '826': 'NYGDPPCAPKDGBR', '842': 'NYGDPPCAPKDUSA'
#         }

#         def get_series_data(series_dict, value_column):
#             all_data = []
#             for country_code, series_id in series_dict.items():
#                 data = fred.get_series(series_id).loc['2010':'2023']
#                 df = data.resample('A').mean().reset_index()
#                 df.columns = ['year', value_column]
#                 df['year'] = df['year'].dt.year
#                 df['country'] = int(country_code)
#                 all_data.append(df)
#             return pd.concat(all_data, ignore_index=True)

#         debt_df = get_series_data(debt_series, 'debt_ratio')
#         gdp_df = get_series_data(gdp_pc_series, 'gdp_per_cap')
#         merged_df = pd.merge(debt_df, gdp_df, on=['year', 'country'])

#         df_trade['period'] = df_trade['period'].astype(int)
#         merged_df['country'] = merged_df['country'].astype(int)
#         df_final = pd.merge(df_trade, merged_df, left_on=['period','reporterCode'], right_on=['year','country'])

#         return df_final[['period','reporterCode','primaryValue','debt_ratio','gdp_per_cap']]





# 2 Plot

In [4]:
# Load from CSV or download fresh data
analyzer = TradeDebtAnalyzer(csv_path='df_final.csv')

# Plot all country-level time series
analyzer.plot()

For countries like US and UK, there seems to be a positive relation between debt to gdp ratio and import value from China.

# 3 Regression
I regress the debt as gdp ratio to import from China, controlling for the country's per capita gdp, time and country fixed effect.

In [5]:
# Run regression
result = analyzer.regress()
print(result.summary)

                            PanelOLS Estimation Summary                            
Dep. Variable:     Debt Ratio (% of GDP)   R-squared:                        0.3070
Estimator:                      PanelOLS   R-squared (Between):             -1.7979
No. Observations:                    244   R-squared (Within):               0.0676
Date:                   Sun, Jun 01 2025   R-squared (Overall):             -1.7467
Time:                           23:42:49   Log-likelihood                   -945.96
Cov. Estimator:                Clustered                                           
                                           F-statistic:                      46.727
Entities:                             18   P-value                           0.0000
Avg Obs:                          13.556   Distribution:                   F(2,211)
Min Obs:                          13.000                                           
Max Obs:                          14.000   F-statistic (robust):            

Using OECD countries in general, the regression says on average, there's no relation between debt to gdp ratio and import from China, given insiginificant coefficient on Import from China.